In [1]:
import os, sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

Investigate the performance of flights over time or simply look at data for a given year and create a graphic that showcases your finding(s).

In [2]:
def addDate(df):
    import datetime 
    #df = df.head(100000)

    dt = df[['Year', 'Month', 'DayofMonth']]
    dt.columns = ["year", "month", "day"]
    dt.head(2)

    #pd.to_datetime(dt["year", "month", "day"])
    df["Date"] = pd.to_datetime(dt.year*10000 + dt.month*100 + dt.day, format='%Y%m%d')
    return df

In [3]:
## From http://www.transtats.bts.gov/ot_delay/OT_DelayCause1.asp?pn=1
## Download file: http://www.transtats.bts.gov/ot_delay/ot_delaycause1.asp?display=download&pn=0&month=6&year=2016

In [4]:
df_delays = pd.read_csv("./rawData/airline_delay_causes.csv")
print(" Dataframe with {0} registers and the folloging data columns: \n {1}".format(
        len(df_delays), ", ".join(df_delays.columns)))

 Dataframe with 55342 registers and the folloging data columns: 
 year,  month, carrier, carrier_name, airport, airport_name, arr_flights, arr_del15, carrier_ct,  weather_ct, nas_ct, security_ct, late_aircraft_ct, arr_cancelled, arr_diverted,  arr_delay,  carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay, Unnamed: 21


In [5]:
## Download yearly files
## http://stat-computing.org/dataexpo/2009/1992.csv.bz2

http://stat-computing.org/dataexpo/2009/the-data.html

In [6]:
df_origs = {}

In [7]:
#years = [1990,1991,1992,1993]
years = range(1987,2009)
print (" Reading ", years[0])

for kyear in years:
    print (" Reading ", kyear)
    if not kyear in df_origs.keys():
        print (" Here ", kyear)
        try:
            print("Try")
            df_origs[kyear] = pd.read_csv("./rawData/{0}.csv.bz2".format(kyear), engine="c")
        except:
            pass

 Reading  1987
 Reading  1987
 Here  1987
Try
 Reading  1988
 Here  1988
Try
 Reading  1989
 Here  1989
Try
 Reading  1990
 Here  1990
Try
 Reading  1991
 Here  1991
Try
 Reading  1992
 Here  1992
Try
 Reading  1993
 Here  1993
Try
 Reading  1994
 Here  1994
Try
 Reading  1995
 Here  1995
Try
 Reading  1996
 Here  1996
Try
 Reading  1997
 Here  1997
Try
 Reading  1998
 Here  1998
Try
 Reading  1999
 Here  1999
Try
 Reading  2000
 Here  2000
Try
 Reading  2001
 Here  2001
Try
 Reading  2002
 Here  2002
Try
 Reading  2003
 Here  2003
Try


/Users/alberto/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 Reading  2004
 Here  2004
Try
 Reading  2005
 Here  2005
Try
 Reading  2006
 Here  2006
Try
 Reading  2007
 Here  2007
Try
 Reading  2008
 Here  2008
Try


In [8]:
#df_origs[2002] = pd.read_csv("./rawData/{0}.csv".format("2002"), engine="c")

In [9]:
df_origs.keys()
sorted(df_origs.keys())

[1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008]

In [10]:
yk = list(df_origs.keys())
print (yk)
df_orig = df_origs[yk[0]]
#df_orig = pd.concat(df_orig,])

df_orig = addDate(df_orig)

print(" Dataframe with {0} registers and the folloging data columns: \n {1}".format(
        len(df_orig), ", ".join(df_orig.columns)))

[1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2003, 2004, 2005, 2006, 2007, 2008]
 Dataframe with 1311826 registers and the folloging data columns: 
 Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, TaxiIn, TaxiOut, Cancelled, CancellationCode, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, Date


# Carriers names

In [230]:
Carriers = set(df_orig["UniqueCarrier"].values.tolist())
print(" Number of carriers {0} (Code):\n\t{1}".format(len(Carriers), ", ".join(Carriers)))

 Number of carriers 14 (Code):
	HP, PS, NW, AS, CO, PA (1), US, WN, AA, PI, TW, EA, DL, UA


In [234]:
def genNamesDict(df_delays):
    cD = df_delays[["carrier","carrier_name"]].drop_duplicates()
    return cD.set_index("carrier").to_dict()["carrier_name"]

cd = genNamesDict(df_delays)
cdl = [cd[i] for i in df_delays["carrier"].drop_duplicates()]
print(" Number of carriers {0} (Names):\n\t{1}".format(len(Carriers), ",\n\t".join(cdl)))

 Number of carriers 14 (Names):
	American Airlines Inc.,
	Alaska Airlines Inc.,
	JetBlue Airways,
	Continental Air Lines Inc.,
	Independence Air,
	Delta Air Lines Inc.,
	ExpressJet Airlines Inc.,
	AirTran Airways Corporation,
	America West Airlines Inc.,
	Envoy Air,
	Northwest Airlines Inc.,
	SkyWest Airlines Inc.,
	ExpressJet Airlines Inc.,
	ATA Airlines d/b/a ATA,
	United Air Lines Inc.,
	US Airways Inc.,
	Southwest Airlines Co.,
	Hawaiian Airlines Inc.,
	Comair Inc.,
	Frontier Airlines Inc.,
	Mesa Airlines Inc.,
	Aloha Airlines Inc.,
	ExpressJet Airlines Inc.,
	Endeavor Air Inc.,
	Virgin America,
	Spirit Air Lines


### Number of flights per year

In [11]:
dd = {}
for kyear in df_origs.keys():
    value = len(df_origs[kyear])
    #print(kyear, value)
    dd[kyear] = value

df = pd.DataFrame(dd, index=[0]).T.reset_index() #, orient='index')
df.columns = ["Year", "Value"]
df.set_index("Year", inplace=True)
df.to_csv("data/01-Year_NFlights.csv")
#df.to_csv
print( df )

        Value
Year         
1987  1311826
1988  5202096
1989  5041200
1990  5270893
1991  5076925
1992  5092157
1993  5070501
1994  5180048
1995  5327435
1996  5351983
1997  5411843
1998  5384721
1999  5527884
2000  5683047
2001  5967780
2003  6488540
2004  7129270
2005  7140596
2006  7141922
2007  7453215
2008  7009728


In [12]:
### Number of flights ON TIME (before scheduled time) per year

In [13]:
df = df_origs[2001]
len(df[df.ArrTime <= df.CRSArrTime])/(float(len(df)))

0.5572645104209606

In [14]:
dd = {}
for kyear in df_origs.keys():
    df = df_origs[kyear]
    value = 1-len(df[df.ArrDelay > 0 ])/(float(len(df)))
    #value = len(df[df.ArrTime <= df.CRSArrTime])/(float(len(df)))
    #print(kyear, value)
    dd[kyear] = value

df = pd.DataFrame(dd, index=[0]).T.reset_index() #, orient='index')
df.columns = ["Year", "Value"]
df.set_index("Year", inplace=True)
df.to_csv("data/02-Year_FlightsOnTime.csv")
#df.to_csv
print( df )

         Value
Year          
1987  0.383918
1988  0.463833
1989  0.438083
1990  0.479961
1991  0.517247
1992  0.506966
1993  0.512849
1994  0.511598
1995  0.500722
1996  0.472055
1997  0.498617
1998  0.533479
1999  0.531440
2000  0.511166
2001  0.591194
2003  0.623746
2004  0.576613
2005  0.571013
2006  0.552310
2007  0.536649
2008  0.574947


## Delay causes

In [15]:
df = df_origs[2004].head(20)
", ".join(df.columns)

'Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, ArrTime, CRSArrTime, UniqueCarrier, FlightNum, TailNum, ActualElapsedTime, CRSElapsedTime, AirTime, ArrDelay, DepDelay, Origin, Dest, Distance, TaxiIn, TaxiOut, Cancelled, CancellationCode, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay'

In [18]:
df = df[df.ArrDelay > 0].copy()
df[["ArrTime","CRSArrTime","Distance","Cancelled"]+[i for i in df.columns if "Delay" in i]]

,ArrTime,CRSArrTime,Distance,Cancelled,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2,920.0,915,599,0,5.0,3.0,0,0,0,0,0
4,918.0,915,599,0,3.0,5.0,0,0,0,0,0
6,944.0,915,599,0,29.0,20.0,20,0,9,0,0
10,923.0,915,599,0,8.0,1.0,0,0,0,0,0
13,939.0,915,599,0,24.0,-5.0,0,0,24,0,0
18,1753.0,1725,1739,0,28.0,5.0,0,0,28,0,0
19,1832.0,1725,1739,0,67.0,65.0,9,0,2,0,56


In [21]:
delaycols = ["CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"]
df[delaycols+["ArrDelay","Cancelled","Diverted"]].sum(axis=0)

CarrierDelay          29.0
WeatherDelay           0.0
NASDelay              63.0
SecurityDelay          0.0
LateAircraftDelay     56.0
ArrDelay             164.0
Cancelled              0.0
Diverted               0.0
dtype: float64

In [25]:
dd = {}
df3 = pd.DataFrame()
for kyear in range(2003,2009): #df_origs.keys():
    df = df_origs[kyear]
    ds = df[delaycols+["ArrDelay", "DepDelay"]].sum(axis=0)
    ds["Year"] = kyear
    df3 = pd.concat([df3, ds], axis=1)
    #print( ds.T )
    #value = len(df[df.ArrTime <= df.CRSArrTime])/(float(len(df)))
    #print(kyear, value)
    #dd[kyear] = value

#df = pd.DataFrame(dd, index=[0]).T.reset_index() #, orient='index')
df = df3.T
df["Year"] = df["Year"].astype(int)
df.set_index("Year",inplace=True)
#df.columns = ["Year", "Value"]

#df.to_csv
print( df ) 

        ArrDelay  CarrierDelay    DepDelay  LateAircraftDelay    NASDelay  \
Year                                                                        
2003  22931403.0     8568280.0  33520821.0         10042720.0  11862557.0   
2004  45476603.0    18833073.0  55258141.0         24566219.0  24472354.0   
2005  50217971.0    21441880.0  60779748.0         26157607.0  24046197.0   
2006  60812894.0    24255294.0  70857247.0         32245816.0  25610715.0   
2007  74151049.0    28808434.0  83127865.0         38004942.0  28200746.0   
2008  55994978.0    24048217.0  68546281.0         31670242.0  26171501.0   

      SecurityDelay  WeatherDelay  
Year                               
2003        80663.0     1987155.0  
2004       179219.0     5037460.0  
2005       141045.0     4713790.0  
2006       221371.0     4853843.0  
2007       176906.0     5739649.0  
2008       114316.0     4633717.0  


In [26]:
sums = df[delaycols].sum(axis=1)
sums.index

for kcol in sums.index: 
    #print(kcol, sums[kcol], df.loc[kcol])
    df.loc[kcol] = df.loc[kcol].div(sums[kcol])
df = df[delaycols].copy()
#df.set_index("Year", inplace=True)
df.columns = [i.replace("Delay","") for i in df.columns]
df.to_csv("data/03-Year_FlightsDelayCauses.csv")
df

,Carrier,Weather,NAS,Security,LateAircraft
Year,,,,,
2003,0.263304,0.061065,0.364538,0.002479,0.308614
2004,0.257676,0.068923,0.334833,0.002452,0.336117
2005,0.280284,0.061618,0.314327,0.001844,0.341927
2006,0.278198,0.055672,0.293745,0.002539,0.369846
2007,0.285428,0.056867,0.279407,0.001753,0.376545
2008,0.277571,0.053484,0.302079,0.001319,0.365547


In [22]:
delaycols = [i for i in df.columns if "Delay" in i]
delaycols.remove("ArrDelay")
delaycols.remove("DepDelay")
df[delaycols+["Cancelled","Diverted"]].sum()
for i in delaycols:
    df[i] = df[i]/df["ArrDelay"]
print(df["ArrDelay"].count())
df["OtherDelay"] = 1 - df[delaycols].sum(axis=1)
df[delaycols+["OtherDelay","Cancelled","Diverted"]].sum()

7


CarrierDelay         0.823984
WeatherDelay         0.000000
NASDelay             2.340196
SecurityDelay        0.000000
LateAircraftDelay    0.835821
OtherDelay           3.000000
Cancelled            0.000000
Diverted             0.000000
dtype: float64

In [101]:
dfDateFinal = pd.DataFrame()
for kyear in range(2003,2009):
    print(kyear)
    df = df_origs[kyear]
    df = addDate(df)

    def top(x):
        x.set_index('Date', inplace=True)
        df = pd.DataFrame({'MayorCause':[],'MaxValue':[]})
        df.index.name='Date'
        df.loc[x.index.values[0],['MayorCause']] = x.sum().nlargest(1).index.tolist()[0]
        df.loc[x.index.values[0],['MaxValue']] = x.sum().nlargest(1).values
        return df

    dfDate = df[["Date"]+delaycols]
    dfDate.columns = [ i.replace("Delay","") for i in dfDate.columns] 
    dfDate = dfDate.groupby('Date').apply(top).reset_index(level=1, drop=True).reset_index()
    dfDate.set_index("Date", inplace=True)
    #dfDate.to_csv("data/04-Date_FlightsDelayCauses.csv")
    #dfDate.head()
    dfDateFinal = pd.concat([dfDateFinal,dfDate], axis=0)
    print (len(dfDateFinal))
    
dfDateFinal.tail(2)

2003
365
2004
731
2005
1096
2006
1461
2007
1826
2008
2192


,MaxValue,MayorCause
Date,,
2003-01-01,0.0,Carrier
2003-01-02,0.0,Carrier


In [138]:
dd = {"Carrier": 1, "NAS":2, "Weather":3, "LateAircraft": 4, "Security": 5}
def ddd(i):
    return dd[i]
dfDateFinal.head()

dfDateFinal["value"] = dfDateFinal["MayorCause"].apply(ddd)
dfDateFinal.to_csv("data/04-Date_FlightsDelayCauses.csv")
dfDateFinal.tail()

,MaxValue,MayorCause,value
Date,,,
2008-12-27,262837.0,LateAircraft,4
2008-12-28,106602.0,LateAircraft,4
2008-12-29,59673.0,Carrier,1
2008-12-30,80162.0,LateAircraft,4
2008-12-31,83589.0,LateAircraft,4


# Airlines 

In [139]:
df_delays.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed:21
0,2003,6,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",752.0,186.0,33.99,27.82,...,17.53,5.0,0.0,8314.0,1367.0,1722.0,3817.0,139.0,1269.0,NaN
1,2003,6,AA,American Airlines Inc.,BOS,"Boston, MA: Logan International",1266.0,225.0,69.43,23.66,...,45.73,7.0,0.0,12139.0,4201.0,1783.0,3067.0,45.0,3043.0,NaN
2,2003,6,AA,American Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",593.0,101.0,17.56,20.49,...,24.69,1.0,1.0,5698.0,1058.0,1332.0,1708.0,0.0,1600.0,NaN
3,2003,6,AA,American Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",203.0,48.0,16.57,9.72,...,9.76,4.0,0.0,3048.0,968.0,742.0,558.0,127.0,653.0,NaN
4,2003,6,AA,American Airlines Inc.,DCA,"Washington, DC: Ronald Reagan Washington National",947.0,147.0,42.88,24.89,...,26.44,5.0,2.0,7868.0,2048.0,1484.0,2767.0,3.0,1566.0,NaN


In [212]:
print(df_delays.columns)
def idfun(i):
    return i
df_delays.columns = [ i.replace(" ", "") for i in df_delays.columns]
df = df_delays.groupby(["carrier", "year"]).mean().reset_index() #level=1, drop=True).reset_index()
print(df.reset_index().head(4))
df = df[["carrier","year"] + [ i for i in df.columns if "_delay" in i ]]
df.columns =  [ i.replace("carrier","name") for i in df.columns ]
df.columns =  [ i.replace("name_delay", "carrier").replace("_delay","") for i in df.columns ]
df.head()

Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'Unnamed:21'],
      dtype='object')
   index carrier  year     month  arr_flights   arr_del15  carrier_ct  \
0      0      9E  2007  6.463087   637.000000  132.859060   29.974832   
1      1      9E  2008  6.761364   537.312500   92.034091   19.628239   
2      2      9E  2009  6.160714   582.976190   95.339286   18.588750   
3      3      9E  2010  6.755319   540.702128  103.994681   20.430957   

   weather_ct     nas_ct  security_ct  late_aircraft_ct  arr_cancelled  \
0    5.633221  50.800805     0.197987         46.252148      22.973154   
1    3.117386  34.643750     0.086875         34.558239      16.721591   
2    3.269226  43.564464     0.102

,name,year,arr,carrier,weather,nas,security,late_aircraft
0,9E,2007,8671.382550,2677.469799,732.214765,2283.892617,6.516779,2971.288591
1,9E,2008,5612.801136,1576.835227,307.477273,1522.295455,3.289773,2202.903409
2,9E,2009,5245.529762,1330.559524,285.946429,1832.803571,2.625000,1793.595238
3,9E,2010,5872.127660,1488.797872,265.468085,1662.505319,6.936170,2448.420213
4,9E,2013,7041.669456,1938.004184,323.401674,2225.702929,7.071130,2547.489540


In [158]:
df.sort_values(["year"]).tail(3)

,carrier,year,arr,carrier,weather,nas,security,late_aircraft
35,AS,2016,2102.073333,580.033333,58.353333,793.433333,11.086667,659.166667
75,DL,2016,16288.804598,6342.379310,933.931034,4043.655172,9.678161,4959.160920
49,B6,2016,10610.611111,2976.972222,324.868056,2724.006944,30.381944,4554.381944


In [172]:
print(len(df), len(df[(df.year<2009) &
                      (df.year>2002)]))
dfU = df[(df.year<2011) & (df.year>2002)]
dfU 

242 118


,name,year,arr,carrier,weather,nas,security,late_aircraft
0,9E,2007,8671.382550,2677.469799,732.214765,2283.892617,6.516779,2971.288591
1,9E,2008,5612.801136,1576.835227,307.477273,1522.295455,3.289773,2202.903409
2,9E,2009,5245.529762,1330.559524,285.946429,1832.803571,2.625000,1793.595238
3,9E,2010,5872.127660,1488.797872,265.468085,1662.505319,6.936170,2448.420213
5,AA,2003,16631.384236,4223.187192,1165.320197,7112.950739,31.014778,4098.911330
6,AA,2004,19866.910920,4542.178161,1537.557471,7832.543103,44.862069,5909.770115
7,AA,2005,18535.836207,4472.741379,1215.916667,6913.839080,21.272989,5912.066092
8,AA,2006,19883.797101,4704.527536,1024.037681,7238.118841,46.530435,6870.582609
9,AA,2007,25509.270833,5991.660714,1674.669643,8550.964286,19.550595,9272.425595
10,AA,2008,22654.321429,6113.741071,890.732143,7797.169643,16.651786,7836.026786


In [183]:
names = set(dfU["name"])
namfli = []
for kname in names: 
    print(len(dfU[ dfU.name == kname] ))
    if len(dfU[ dfU.name == kname] ) == 8:
        namfli.append(kname)

namfli 

8
3
8
7
4
3
8
8
8
8
8
8
3
6
8
8
4
4
8
7
5
5
8
8


['FL', 'AS', 'MQ', 'UA', 'EV', 'US', 'WN', 'HA', 'B6', 'OO', 'AA', 'CO', 'DL']

In [249]:
dff = pd.DataFrame()
for kname in namfli:
    dftmp = dfU[dfU.name == kname][["year", "late_aircraft"]].set_index("year")
    dftmp.columns = [kname]
    #print(dftmp)
    #dff = pd.concat([dff, dfU[dfU.name == kname][["name", "year","late_aircraft"]]], axis=0)
    dff = pd.concat([dff, dftmp], axis=1)

dff = dff/dff.loc[2003]

cd = genNamesDict(df_delays)
dff.columns = [str(cd[i]) for i in dff.columns]
dff.to_csv("data/05-DelayAircraftCarrier.csv")
dff.head()

,AirTran Airways Corporation,Alaska Airlines Inc.,Envoy Air,United Air Lines Inc.,ExpressJet Airlines Inc.,US Airways Inc.,Southwest Airlines Co.,Hawaiian Airlines Inc.,JetBlue Airways,SkyWest Airlines Inc.,American Airlines Inc.,Continental Air Lines Inc.,Delta Air Lines Inc.
year,,,,,,,,,,,,,
2003,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2004,1.027819,1.315781,1.144472,1.181757,0.372753,1.101932,1.496053,0.814863,1.582610,1.722558,1.441790,1.107386,1.537024
2005,1.692860,1.705926,1.192199,1.361266,0.402873,1.347652,1.606966,1.297914,3.544991,2.105736,1.442350,1.311687,1.774582
2006,1.649684,1.465138,1.659607,1.911773,0.684616,1.307103,1.758336,2.962625,4.496346,4.173463,1.676197,2.047832,1.537323
2007,1.408343,1.610268,2.175140,2.311085,1.238614,1.864045,1.768046,3.449370,5.599559,4.292704,2.262168,2.149492,1.524659


In [248]:
dff = pd.DataFrame()
for kname in namfli:
    dftmp = dfU[dfU.name == kname][["year", "arr"]].set_index("year")
    dftmp.columns = [kname]
    #print(dftmp)
    #dff = pd.concat([dff, dfU[dfU.name == kname][["name", "year","late_aircraft"]]], axis=0)
    dff = pd.concat([dff, dftmp], axis=1)

dff = dff/dff.loc[2003]

cd = genNamesDict(df_delays)
dff.columns = [str(cd[i]) for i in dff.columns]
dff.to_csv("data/06-DelayTotalCarrier.csv")
dff.head()

,AirTran Airways Corporation,Alaska Airlines Inc.,Envoy Air,United Air Lines Inc.,ExpressJet Airlines Inc.,US Airways Inc.,Southwest Airlines Co.,Hawaiian Airlines Inc.,JetBlue Airways,SkyWest Airlines Inc.,American Airlines Inc.,Continental Air Lines Inc.,Delta Air Lines Inc.
year,,,,,,,,,,,,,
2003,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2004,1.046357,1.277949,1.094236,1.134945,1.106513,1.115708,1.455292,0.747727,1.340056,1.405029,1.194543,1.091024,1.379240
2005,1.560324,1.722361,1.175580,1.089476,1.064566,1.350506,1.600971,0.755120,2.791234,1.602374,1.114510,1.227482,1.423765
2006,1.389585,1.483484,1.526620,1.354697,1.297763,1.453231,1.767338,1.138705,3.796310,2.239641,1.195559,1.581846,1.148358
2007,1.190326,1.653602,1.967393,1.540971,1.560399,1.938567,1.754456,1.396760,4.881482,2.029741,1.533803,1.618100,1.124227


# Data

In [50]:
def defineDataFrame(dff, selK, keep=[]):
    
    dfw = dff.copy()
    print(dfw.columns)
    if not keep == []:
        # Keep indexes
        keep = keep + selK 
        for kcol in dfw.columns:
            if kcol not in keep:
                #print("Removing ", kcol)
                dfw.drop(kcol, inplace=True, axis=1)
    print("Counting")
    grBCou = dfw.groupby(by=selK).count() #.unstack()
    print("Summing")
    grBSum = dfw.groupby(by=selK).sum()
    print("Averaging")
    grBAvg = dfw.groupby(by=selK).mean()
    #grBCou.reset_index()

    dfr = grBAvg.join(
            grBSum, how="outer", 
            rsuffix="_sum").join(
            grBCou, how="outer", 
            lsuffix="_avg", rsuffix="_count").reset_index()

    #print(dfr.columns)
    return dfr

In [51]:
df = df_orig.copy()

In [52]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'Date'],
      dtype='object')

In [74]:
kl = [ i for i in df.columns if "Delay" in i ]
print(kl)
dfAApre = df.head(8000)[["Origin", "Dest", "Cancelled", "ActualElapsedTime", "CRSElapsedTime"]+kl]
dfAApre.head(8)

['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


,Origin,Dest,Cancelled,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,ORD,CLT,0,98.0,105,-14.0,-7.0,0,0,0,0,0
1,ORD,CLT,0,110.0,105,-4.0,-9.0,0,0,0,0,0
2,ORD,CLT,0,107.0,105,5.0,3.0,0,0,0,0,0
3,ORD,CLT,0,92.0,105,-16.0,-3.0,0,0,0,0,0
4,ORD,CLT,0,103.0,105,3.0,5.0,0,0,0,0,0
5,ORD,CLT,0,97.0,105,-10.0,-2.0,0,0,0,0,0
6,ORD,CLT,0,114.0,105,29.0,20.0,20,0,9,0,0
7,ORD,CLT,0,97.0,105,-11.0,-3.0,0,0,0,0,0


## Data:: Airport to airport Flights

In [80]:
selK = ["Origin", "Dest"]
delays = [kl]
grBAvg = dfAApre.groupby(by=selK).mean()
grBAvg.reset_index().to_csv("data/Flights-OrigDest-Mean.csv")

dfAA = grBAvg.reset_index()
#dfAA = defineDataFrame(df, selK, df.columns)

In [93]:
Airports = (set(list(dfAA["Origin"].values)+list(dfAA["Dest"].values)))
", ".join(Airports)

'IAH, MHT, BOI, OAK, CLT, TUS, ALB, GEG, ICT, BTV, BUF, SNA, OMA, DTW, MSY, STL, ATL, EWR, IND, BWI, DCA, SLC, JAC, MDT, LAX, FLL, PBI, SAT, TPA, CVG, IAD, LAS, MCI, ABQ, ONT, DEN, BOS, MSP, TUL, CMH, OKC, PHL, BDL, ORD, DSM, RIC, SEA, GSO, PDX, MIA, DFW, PVD, BIL, BUR, COS, MCO, PIT, LGA, SJC, SMF, CLE, PHX, GRR, SAN, SFO'

In [94]:
dfAA.to_csv("data/fligths-DateCancelled.csv")


In [155]:
dfAAA = dfAA.set_index(["Origin","Dest"])
dd = {}
for i,j in dfAAA.index:
    #print(i, j)
    
    try:
        dd[i]["imports"] = dd[i]["imports"] + [j]
        dd[i]["dada"] = dd[i]["dada"] + [j]
    except:
        dd[i] = {} 
        dd[i]["imports"] = [j]
        dd[i]["dada"] = [j]
dd 

{'ABQ': {'dada': ['DEN'], 'imports': ['DEN']},
 'ALB': {'dada': ['ORD'], 'imports': ['ORD']},
 'ATL': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'BDL': {'dada': ['IAD'], 'imports': ['IAD']},
 'BIL': {'dada': ['DEN'], 'imports': ['DEN']},
 'BOI': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'BOS': {'dada': ['DEN', 'IAD', 'ORD'], 'imports': ['DEN', 'IAD', 'ORD']},
 'BUR': {'dada': ['DEN'], 'imports': ['DEN']},
 'BWI': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'CLE': {'dada': ['ORD'], 'imports': ['ORD']},
 'CLT': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'CMH': {'dada': ['DEN', 'ORD'], 'imports': ['DEN', 'ORD']},
 'DCA': {'dada': ['ORD'], 'imports': ['ORD']},
 'DEN': {'dada': ['ATL',
   'BOI',
   'CLE',
   'CLT',
   'CMH',
   'COS',
   'DFW',
   'DSM',
   'DTW',
   'GEG',
   'IAD',
   'IAH',
   'ICT',
   'IND',
   'JAC',
   'LAS',
   'LAX',
   'MCI',
   'MIA',
   'MSP',
   'OAK',
   'OMA',
   'ONT',
   'ORD',
   'PDX',
   'PHL',
   'PHX',
   'SE

In [166]:
dfAAAA = pd.DataFrame(dd).T
dfAAAA.index.name = "name"
dfAAAA.head(2)

,dada,imports
name,,
ABQ,[DEN],[DEN]
ALB,[ORD],[ORD]


In [170]:
dfAAAA.reset_index().to_json("data/Airports-flare.json", orient="records")

In [176]:
dfAAA.reset_index().to_json("data/Airports-flare2.json", orient="records")

In [123]:
for i, j in dfAAA.index:
    idx = (i,j)
    print(idx)
    print(dfAAA.loc[idx])

('ABQ', 'DEN')
Cancelled             0.00
ActualElapsedTime    75.48
CRSElapsedTime       76.00
ArrDelay             -1.24
DepDelay             -0.72
CarrierDelay          3.44
WeatherDelay          0.00
NASDelay              0.40
SecurityDelay         0.00
LateAircraftDelay     0.00
Name: (ABQ, DEN), dtype: float64
('ALB', 'ORD')
Cancelled              0.000000
ActualElapsedTime    165.333333
CRSElapsedTime       136.000000
ArrDelay              94.833333
DepDelay              65.500000
CarrierDelay           0.000000
WeatherDelay           0.000000
NASDelay              57.333333
SecurityDelay          0.000000
LateAircraftDelay     37.500000
Name: (ALB, ORD), dtype: float64
('ATL', 'DEN')
Cancelled              0.000000
ActualElapsedTime    198.612903
CRSElapsedTime       197.806452
ArrDelay               7.516129
DepDelay               6.709677
CarrierDelay           5.693548
WeatherDelay           0.000000
NASDelay               2.629032
SecurityDelay          0.000000
LateAircraf

## Data:: Date vs Cancelled Flights

In [ ]:
selK = ["Date"]
delays = ["Cancelled"]
df1 = defineDataFrame(df, selK, delays)

df1.head(2)

In [ ]:
### Date vs mean delay 
df1.set_index("Date").to_csv("data/fligths-DateCancelled.csv")

## Data:: Date vs Delayed Flights

In [ ]:
selK = ["Date"]
delays = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
df2 = defineDataFrame(df, selK, delays)

df2.head(2)

In [ ]:
df2.set_index("Date").to_csv("data/fligths-DateDelayed.csv")

## Data:: Year vs Delayed Flights

In [ ]:
selK = ["Year"]
delays = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
df3 = defineDataFrame(df, selK, delays)

df3.head(2)

In [ ]:
df3.set_index(selK).to_csv("data/fligths-YearDelayed.csv")

In [ ]:
## Data:: Year vs Delayed Flights

In [ ]:
selK = ["year"]
delays = ["arr_delay",  "carrier_delay", "weather_delay", "nas_delay", "security_delay", "late_aircraft_delay"]

In [ ]:
df_delays.columns 

In [ ]:
df4 = defineDataFrame(df_delays, selK, delays)
df4.head(2)

In [ ]:
df4.set_index(selK).to_csv("data/flights-YearDelayed.csv")